# Data requirements and preprocessing

## We require three types of data: Mutation, Copy Number, and Response (the label).

Here, as an example, we preprocess data from Nguyen et al., Cell 2022, which contains a comprehensive dataset; however, we only require the following three data types:

① data_mutation.txt
② data_cna.txt
③ data_clinical_sample.txt


Data Link：https://www.cbioportal.org/study/summary?id=msk_met_2021

### 1. Mutation ———— data_mutation.txt

In [1]:
import pandas as pd
from os.path import join

data_mutations_filename = 'data_mutations.txt'

mutations = pd.read_csv(data_mutations_filename, sep='\t', low_memory=False, skiprows=0)

id_col = 'Tumor_Sample_Barcode'

mutations['Variant_Classification'].value_counts()

#exclude = ['Silent', 'Intron', "3\'UTR", "5\'UTR", 'RNA', 'lincRNA']
exclude = ['Silent', 'Intron', "5\'UTR"]

mutations = mutations[~mutations['Variant_Classification'].isin(exclude)].copy()

mutations_table = pd.pivot_table(data=mutations, index=id_col, columns='Hugo_Symbol', values='Variant_Classification',aggfunc='count')

mutations_table = mutations_table.fillna(0)

total_numb_mutations = mutations_table.sum().sum()
number_samples = mutations_table.shape[0]

print('number of mutations:', total_numb_mutations, 'average mutation/sample:', total_numb_mutations / (number_samples + 0.0))

mutations_table.to_csv('msk_2021_mut.csv')

number of mutations: 230242.0 average mutation/sample: 9.301579606512341


### 2. Copy Number ———— data_cna.txt

In [2]:
data_copy_number_filename = 'data_cna.txt'
cna = pd.read_csv(data_copy_number_filename, sep='\t', low_memory=False, skiprows=0, index_col=0)

cna = cna.T
cna = cna.fillna(0.)

cna.to_csv('msk_2021_cna.csv')

### 3. Response ———— data_clinical_sample.txt

In [3]:
data_response_filename = 'data_clinical_sample.txt'

data_response = pd.read_csv(data_response_filename, sep='\t', low_memory=False, skiprows=4)

response = data_response[['SAMPLE_ID', 'SAMPLE_TYPE']]
response = response.rename(columns={"SAMPLE_ID": "id", "SAMPLE_TYPE": "response"})

response['response'] = response['response'].replace('Metastasis', 1)
response['response'] = response['response'].replace('Primary', 0)
response = response.drop_duplicates()

response.to_csv('msk_2021_response.csv')

### 4. Select col

In [2]:
import pandas as pd 
import numpy as np

mut = ['TP53','AR','PTEN','NUP98','OBSCN','MUC16','PSMD1','ABCA13','PPP2R5D','RB1','ATM','CREBBP','COL11A1','EP300','RANBP2','ROCK1','DCTN1','TP53BP1','NUP205','AGL']
cn = ['AR','PTEN','CUL1','PSMD2','RB1','PPP2R2A','PDGFA','PSAP','UBE2W','MDM2','ANAPC16','MDM4','PIK3CB','EIF3E','RAC1','COL1A2','FGFR1','MAP3K7','HIST1H2BD','HIST1H2AC','AP2A2','RRAGD','NEURL1B','NBN','HIST1H2AB','RPS27','GOLGA7','PSMD14','WWTR1']

mut_data = pd.read_csv('msk_2021_mut.csv',index_col=0)
cn_data = pd.read_csv('msk_2021_cna.csv',index_col=0)

mut_selection = mut_data.reindex(columns=mut, fill_value=0)
cn_selection = cn_data.reindex(columns=cn,fill_value=0)

mut_selection.to_csv('msk_2021_mut_138.csv')
cn_selection.to_csv('msk_2021_cna_138.csv')

### 5. data_all

In [1]:
from data_access import Data

import numpy as np
import pandas as pd
from os.path import join, dirname, realpath

from sklearn.model_selection import train_test_split

# 1. data splits
# data-paper
response = pd.read_csv('msk_2021_response.csv')

# 

current_dir = dirname(realpath('processed'))
output_dir = 'splits'
output_dir = join(current_dir, output_dir)


all_ids = response
ids = response['id'].values
y = response['response'].values
#first split train and test
ids_train, ids_test, y_train, y_test = train_test_split(ids, y, test_size=0.1, stratify=y, random_state=42)
#then in train set, split train and validate

ids_train, ids_validate, y_train, y_validate = train_test_split(ids_train, y_train, test_size=len(y_test),
                                                                stratify=y_train, random_state=42)

test_set = pd.DataFrame({'id': ids_test, 'response': y_test})
train_set = pd.DataFrame({'id': ids_train, 'response': y_train})
validate_set = pd.DataFrame({'id': ids_validate, 'response': y_validate})

print(test_set.response.value_counts() / float(test_set.response.value_counts().sum()))
print(train_set.response.value_counts() / float(train_set.response.value_counts().sum()))
print(validate_set.response.value_counts() / float(validate_set.response.value_counts().sum()))

test_set.to_csv(join(output_dir, 'test_set.csv'))

validate_set.to_csv(join(output_dir, 'validation_set.csv'))
train_set.to_csv(join(output_dir, 'training_set.csv'))

total_number_samples = train_set.shape[0]
number_patients = np.geomspace(100, total_number_samples, 20)
number_patients = [int(s) for s in number_patients][::-1]
print(number_patients)

for i, n, in enumerate(number_patients):
    if i == 0:
        filename = join(output_dir, 'training_set_0.csv')
        train_set.to_csv(filename)
        continue
    number_samples = n
    print(i, number_samples)
    print(ids_train.shape, y_train.shape)
    ids_train, ids_validate, y_train, y_validate = train_test_split(ids_train, y_train, train_size=n,
                                                                    stratify=y_train, random_state=422342)
    print(ids_train.shape, y_train.shape)
    train_set = pd.DataFrame({'id': ids_train, 'response': y_train})
    filename = join(output_dir, 'training_set_{}.csv'.format(i))
    train_set.to_csv(filename)


# 2. feature_engineering
selected_genes = 'tcga_prostate_expressed_genes_and_cancer_genes.csv'

data = Data(id='ALL', type='prostate_paper', params={'data_type': ['mut_important', 'cnv_del', 'cnv_amp'], 'selected_genes': selected_genes})

x_train, x_test, y_train, y_test, info_train, info_test, columns = data.get_train_test()

# -------------------------  x  --------------------------
x = np.concatenate((x_train, x_test))

# data prad_msk_stopsack_2021
np.savetxt('data_all.csv', x, delimiter=',', fmt='%f')

# -------------------------  y  --------------------------
y = np.concatenate((y_train, y_test))

# data prad_msk_stopsack_2021
np.savetxt('y_true_all.csv', y, delimiter=',', fmt='%f')

response
0    0.606672
1    0.393328
Name: count, dtype: float64
response
0    0.606479
1    0.393521
Name: count, dtype: float64
response
0    0.606284
1    0.393716
Name: count, dtype: float64
[20619, 15576, 11766, 8889, 6715, 5072, 3832, 2894, 2186, 1652, 1248, 942, 712, 538, 406, 307, 231, 175, 132, 100]
1 15576
(20619,) (20619,)
(15576,) (15576,)
2 11766
(15576,) (15576,)
(11766,) (11766,)
3 8889
(11766,) (11766,)
(8889,) (8889,)
4 6715
(8889,) (8889,)
(6715,) (6715,)
5 5072
(6715,) (6715,)
(5072,) (5072,)
6 3832
(5072,) (5072,)
(3832,) (3832,)
7 2894
(3832,) (3832,)
(2894,) (2894,)
8 2186
(2894,) (2894,)
(2186,) (2186,)
9 1652
(2186,) (2186,)
(1652,) (1652,)
10 1248
(1652,) (1652,)
(1248,) (1248,)
11 942
(1248,) (1248,)
(942,) (942,)
12 712
(942,) (942,)
(712,) (712,)
13 538
(712,) (712,)
(538,) (538,)
14 406
(538,) (538,)
(406,) (406,)
15 307
(406,) (406,)
(307,) (307,)
16 231
(307,) (307,)
(231,) (231,)
17 175
(231,) (231,)
(175,) (175,)
18 132
(175,) (175,)
(132,) (132,)
19 10

In [2]:
x.shape

(24753, 138)